In [58]:
import mxnet as mx
import gluonnlp as nlp

import time
import random
import os
import io
import logging
import numpy as np
import sacremoses
from tqdm import tqdm

# Local Libraries
import nmt
import dataprocessor
import utils
import nmt.gnmt_hparams

# Seeds for reproducibility
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)

ctx = mx.gpu(0)

In [2]:
# Dataset Parameters
dataset = 'WMT2014'
src_lang, tgt_lang = 'en', 'de'
# No limit on sentences length
src_max_len, tgt_max_len = -1, -1

In [3]:
# WMT2016 Dataset (Train and Evaluation)
wmt_train_text_bpe = nlp.data.WMT2016BPE("train", # BPE: cheapest --> cheap@@, est
                                         src_lang=src_lang,
                                         tgt_lang=tgt_lang)

wmt_train_text     = nlp.data.WMT2016("train",
                                      src_lang=src_lang,
                                      tgt_lang=tgt_lang)

wmt_test_text_bpe  = nlp.data.WMT2016BPE("newstest2016", # BPE: cheapest --> cheap@@, est
                                         src_lang=src_lang,
                                         tgt_lang=tgt_lang)

wmt_test_text      = nlp.data.WMT2016("newstest2016",
                                     src_lang=src_lang,
                                     tgt_lang=tgt_lang)

wmt_src_vocab = wmt_train_text_bpe.src_vocab
wmt_tgt_vocab = wmt_train_text_bpe.tgt_vocab

/home/andreto/.local/lib/python3.7/site-packages/gluonnlp/vocab/vocab.py:591: UserWarning: Detected a corrupted index in the deserialize vocabulary. For versions before GluonNLP v0.7 the index is corrupted by specifying the same token for different special purposes, for example eos_token == padding_token. Deserializing the vocabulary nevertheless.
  'Detected a corrupted index in the deserialize vocabulary. '


In [4]:
# Processing datasets
wmt_train_text_bpe = mx.gluon.data.SimpleDataset(wmt_train_text_bpe)
wmt_train_text     = mx.gluon.data.SimpleDataset(wmt_train_text)
wmt_test_text_bpe  = mx.gluon.data.SimpleDataset(wmt_test_text_bpe)
wmt_test_text      = mx.gluon.data.SimpleDataset(wmt_test_text)

In [5]:
# Dataset example (human-readable): English and German
print(wmt_test_text[16][0])
print(wmt_test_text[16][1])

By the end of the day, there would be one more death: Lamb took his own life as police closed in on him.
Bis zum Ende des Tages gab es einen weiteren Tod: Lamm nahm sich das Leben, als die Polizei ihn einkesselte.


In [22]:
# Retrieve (split) translated sequences (target)
wmt_train_tgt_sentences = wmt_train_text.transform(lambda src, tgt: tgt)
wmt_test_tgt_sentences  = wmt_test_text.transform(lambda src, tgt: tgt)
print("Sample target sentence:")
print(wmt_test_tgt_sentences[16])

Sample target sentence:
Bis zum Ende des Tages gab es einen weiteren Tod: Lamm nahm sich das Leben, als die Polizei ihn einkesselte.


In [7]:
# Dataset processing: clipping, tokenizing, indexing and adding of EOS (src/tgt) / BOS (tgt)
wmt_transform_fn = dataprocessor.TrainValDataTransform(wmt_src_vocab, wmt_tgt_vocab)

wmt_train_processed = wmt_train_text_bpe.transform(wmt_transform_fn, lazy=False)
wmt_test_processed  = wmt_test_text_bpe.transform(wmt_transform_fn, lazy=False)

wmt_train_text_with_len = wmt_train_processed.transform(nmt.utils.get_length_index_fn(), lazy=False)
wmt_test_text_with_len  = wmt_test_processed.transform(nmt.utils.get_length_index_fn(), lazy=False)

print(wmt_test_text_with_len[16][0])
print(wmt_test_text_with_len[16][1])

[ 2083 28753 16760 23875 28753 15230    28 28783 31223 12931 24017 23247
 15259   569  5971 12813 29083 20097 24348 22312 12290 24829 14439 20585
 24004 20061    62     3]
[    2  1897 31601  3259 15535  9414 18646 17382 16407 30851  9629   569
  5971 22642 23439 27119 15199  6041    28 11681 15681  7670 20454 16394
 21488 26868 28535    62     3]


In [8]:
# Batcher
wmt_batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(),                   # Source Token IDs
    nlp.data.batchify.Pad(),                   # Target Token IDs
    nlp.data.batchify.Stack(dtype='float32'),  # Source Sequence Length
    nlp.data.batchify.Stack(dtype='float32'),  # Target Sequence Length
    nlp.data.batchify.Stack())                 # Index

/home/andreto/.local/lib/python3.7/site-packages/gluonnlp/data/batchify/batchify.py:235: UserWarning: Padding value is not given and will be set automatically to 0 in data.batchify.Pad(). Please check whether this is intended (e.g. value of padding index in the vocabulary).
  'Padding value is not given and will be set automatically to 0 '


In [45]:
# Hyperparameters
hparams = nmt.gnmt_hparams

In [9]:
# Samplers
wmt_train_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=wmt_train_text_with_len.transform(lambda src, tgt, src_len, tgt_len, idx: (src_len, tgt_len)),
    num_buckets=hparams.num_buckets,
    batch_size=hparams.batch_size)
print(wmt_train_batch_sampler.stats())

wmt_test_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=wmt_test_text_with_len.transform(lambda src, tgt, src_len, tgt_len, idx: (src_len, tgt_len)),
    num_buckets=hparams.num_buckets,
    batch_size=hparams.test_batch_size)
print(wmt_test_batch_sampler.stats())

FixedBucketSampler:
  sample_num=4500966, batch_num=2250484
  key=[(163, 115), (324, 227), (485, 339)]
  cnt=[4497590, 3303, 73]
  batch_size=[2, 2, 2]
FixedBucketSampler:
  sample_num=2999, batch_num=1501
  key=[(37, 42), (70, 78), (103, 114)]
  cnt=[2353, 619, 27]
  batch_size=[2, 2, 2]


In [10]:
# DataLoaders
wmt_train_data_loader = mx.gluon.data.DataLoader(
    wmt_train_text_with_len,
    batch_sampler=wmt_train_batch_sampler,
    batchify_fn=wmt_batchify_fn,
    num_workers=8)
print('Number of training batches:', len(wmt_train_data_loader))

wmt_test_data_loader = mx.gluon.data.DataLoader(
    wmt_test_text_with_len,
    batch_sampler=wmt_test_batch_sampler,
    batchify_fn=wmt_batchify_fn,
    num_workers=8)
print('Number of testing batches:', len(wmt_test_data_loader))

Number of training batches: 2250484
Number of testing batches: 1501


In [12]:
# Model
encoder, decoder, one_step_ahead_decoder = nmt.gnmt.get_gnmt_encoder_decoder(
    hidden_size=hparams.num_hidden,
    dropout=hparams.dropout,
    num_layers=hparams.num_layers,
    num_bi_layers=hparams.num_bi_layers)

gnmt_model = nlp.model.translation.NMTModel(
    src_vocab=wmt_src_vocab,
    tgt_vocab=wmt_tgt_vocab,
    encoder=encoder,
    decoder=decoder,
    one_step_ahead_decoder=one_step_ahead_decoder,
    embed_size=hparams.num_hidden,
    prefix='gnmt_')

gnmt_model.initialize(init=mx.init.Uniform(0.1), ctx=ctx)
static_alloc = True
gnmt_model.hybridize(static_alloc=static_alloc)

In [13]:
scorer=nlp.model.BeamSearchScorer(
    alpha=hparams.lp_alpha,
    K=hparams.lp_k)

gnmt_translator = nmt.translation.BeamSearchTranslator(
    model=gnmt_model,
    beam_size=hparams.beam_size,
    scorer=scorer,
    max_length=tgt_max_len + 100)

print("Use beam_size={}, alpha={}, K={}".format(hparams.beam_size, hparams.lp_alpha, hparams.lp_k))

Use beam_size=10, alpha=1.0, K=5


In [23]:
# Evaluation
eval_start_time = time.time()
wmt_loss_function = nlp.loss.MaskedSoftmaxCELoss()
wmt_loss_function.hybridize()
wmt_detokenizer = nlp.data.SacreMosesDetokenizer()

gnmt_test_loss, gnmt_test_translation_out = nmt.utils.evaluate(
    gnmt_model,
    wmt_test_data_loader,
    wmt_test_loss_function,
    gnmt_translator,
    wmt_tgt_vocab,
    wmt_detokenizer,
    ctx)

gnmt_test_bleu_score, _, _, _, _ = nmt.bleu.compute_bleu(
    [wmt_test_tgt_sentences],
    gnmt_test_translation_out,
    tokenized=False,
    tokenizer=hparams.bleu,
    split_compound_word=False,
    bpe=False)

print('WMT16 EN-DE SOTA model test loss: %.2f; test bleu score: %.2f; time cost %.2fs' %(gnmt_test_loss, gnmt_test_bleu_score * 100, (time.time() - eval_start_time)))

100%|██████████| 1501/1501 [11:40<00:00,  2.14it/s]


WMT16 EN-DE SOTA model test loss: 9.06; test bleu score: 0.00; time cost 706.79s


In [50]:
nmt.gnmt_hparams.beam_size

10

In [71]:
# Training
trainer = mx.gluon.Trainer(gnmt_model.collect_params(), 'adam', {'learning_rate': hparams.lr})

In [87]:
nmt.utils.train(
    gnmt_model,
    wmt_train_data_loader,
    wmt_test_data_loader,
    wmt_loss_function,
    trainer,
    gnmt_translator,
    wmt_tgt_vocab,
    wmt_detokenizer,
    hparams.save_dir,
    hparams,
    ctx)

  0%|          | 10/2250484 [00:09<728:36:31,  1.17s/it]

[Epoch 0 Batch 10/2250484] loss=6.4054, ppl=605.0831, gnorm=3.6659, throughput=0.79K wps, wc=7.86K



  0%|          | 20/2250484 [00:19<647:04:45,  1.04s/it]

[Epoch 0 Batch 20/2250484] loss=9.5011, ppl=13375.0654, gnorm=2.5180, throughput=0.81K wps, wc=7.71K



  0%|          | 30/2250484 [00:26<465:20:26,  1.34it/s]

...

[Epoch 0 Batch 110/2250484] loss=8.7276, ppl=6171.1156, gnorm=0.8806, throughput=0.85K wps, wc=4.16K



  0%|          | 120/2250484 [01:17<273:27:04,  2.29it/s]

[Epoch 0 Batch 120/2250484] loss=8.5823, ppl=5336.3750, gnorm=1.0365, throughput=0.73K wps, wc=3.64K



  0%|          | 0/1 [01:17<?, ?it/s]


KeyboardInterrupt: 

  0%|          | 0/2250484 [00:00<?, ?it/s]

5
(
[[  617 10790   638  9378  6311  9368  1430 10790 10794  7187   617 10790
   7168  9384  3038  7337  5929 10791  3038  7182   616   589  2152  9809
   9368  9393   589  2998   587  5934  9390 10794  9390  2557   615   599
   2986 10408  7188  5927  2553  7173 10409  2117  7939  3595  7187  9809
   2570  6303  2986  9385  5935  4098  5023  4993   640  7168  7350   641
   7173 10408  7188  5927  2553  7173 10409  2117  7939  3595  5014   574
   9987  7177   641  7173  2162  4993  4998   606   618  2141  9987  5924
   5929 10419   618    28  8356  4630  9804  2553  8356  9801  4614   616
    638  9378  6311  9368  1438  6285   595  2986  7187   630  8356  5020
    617  2134  3585  8356  9801  4614   616   638  9374  5432  1438  7373
   9981  4975  3009    28  1445  4960 10815 10815   624  2553  3031  9383
   9373  4998  6285  3033  7945  3039  9981  3041  9385  7937  4981  4637
   9390  8367  2998  5432  2557   615   599  3041  3574  7951   617 10790
   8356  9801  4614  9825  3038  9

NameError: name 'src_seq' is not defined

In [85]:
reload(nmt)
reload(nmt.utils)
reload(nmt.gnmt_hparams)

hparams = nmt.gnmt_hparams

In [86]:
wmt_loss_function = nlp.loss.MaskedSoftmaxCELoss()
wmt_loss_function.hybridize()
wmt_detokenizer = nlp.data.SacreMosesDetokenizer()